# how to optimize GEMM note

## summary

高性能计算的关键在于提高计算时间与移动数据时间的比值。也就是说我们希望移动一定数据后，能够使用这些数据计算尽量多次。这也就是所谓的复用。
按照[gotopaper](https://www.cs.utexas.edu/users/pingali/CS378/2008sp/papers/gotoPaper.pdf)中的说法，使得计算达到浮点峰值性能的关键就在于，在当下的计算循环内，计算所需的数据都正好（在计算过程中）处于cache中，那么cpu不需要停下来去主存中申请数据，能够一直不停地毫无怨言地计算。或者换一种方向来表达：在固定的cache大小下，往cache中存入数据，使得在数据停留在cache中时，尽量完成更多的计算。（反正来都来了，能多算就多算点）   
而达到这一点的方法就是分块。为什么要分块呢？  
最直观的矩阵计算的方法是内积（dot），内积计算矩阵C的一个块需要相应的A的一行（与块的大小对应的行）以及B的一列，并且他们都需要完整的访问完矩阵的行或者列。~~并且数据之间的对应非常少（对应的意识是，计算中需要这个元素的次数少）~~。内积方式的劣势在于，~~在计算期间需要将C block保留在cache当中~~，但是C block并没有实际参与计算。同时在计算时，需要同时streaming A与B，这就使得bandwidth减半。  
所以选择另一种计算方式，外积叠加或者rank 1 update。通过外积来streaming C与A或者C与B中的一个（以下讨论straming C AND B),这样保证了A停留在cache中，并且相同的A的数据只来到cache中一次。但是实际上为了计算的效率，进一步选择B的分块使得B停留在cache当中，这样计算所需的数据就都停留在cache当中。而C不停的streaming from main memory to add result。cache分为三级，经过实验（别人的实验），当A停留在L2cache当中时，访问它的性能最高（这里应该是考虑访问速度与内存含量的trade off，虽然访问L1更快，但是L1装的数据更少）。而更加大的B block就选择放置在L3 cache当中。能够达成这个效果的block方式如下：  
  
<img src="https://www.cs.utexas.edu/users/flame/laff/pfhp/images/Week3/BLISPicturePack.png" style="zoom:40%" />   
  
  
在我现在看过的blas相关的文章当中，暂时没有看过优于甚至实质上不同于这个方式的blocking方式。   
采用这个方法的要点在于选择MC, KC, NC的。根据具体的芯片型号（可以设置自动获取芯片型号并且计算动态分块大小吗？为什么没有人做这个？）选择合适大小的MC, KC, NC，并且根据它们设置循环，来鼓励数据停留在我们希望它停留的cache当中（毕竟我们没法指定某些数据停留在哪里）。   

#### BLOCK SIZE
gotopaper中提到了选择MC, KC, NC的以下原则：  
- 1. mc * kc 要小，小到 『 A + B的 nr 列 + C 的 nr 列 』能够一起塞进 cache  
- 2. 如果 1. 被满足，CPU 计算时不再受内存速度的限制，即得到的gflops值就是真实的计算能力  
- 3. A 或 A 的分块只会被加载进 Cache 一次，gemm过程中不会被换入又换出  
- 4. mc 和 kc 要小，小到 『 A + B的 nr 列 + C 的 nr 列 』能够被 TLB 索引，即一定是小于 L2 cache 的。  
- 5. A 或 A 的分块只被加载到 L2 cache 一次    

[这里](https://zhuanlan.zhihu.com/p/69700540#:~:text=mr%20%E7%9A%84%E5%A4%A7%E5%B0%8F%EF%BC%89%E3%80%82-,GEPB%20%E7%BA%A6%E6%9D%9F%E6%9D%A1%E4%BB%B6,-%E6%88%91%E4%BB%AC%E8%80%83%E5%AF%9F%E6%9C%80%E5%90%8E)提到了一些约束原则，我感觉其中前后的逻辑并不是很严谨，它这些约束的推理貌似并不是在同一个假设背景下做出的。其中比较有用的部分是：  
- 因为计算用的数据都在L1Cache，所以每次从 PanelA 加载的 nr 行不宜超过 L1Cache大小。
    - 最里层循环计算所需的数据都会load 到L1cache中，到时候需要考虑L1cache的容量限制。
- L1Cache miss 可以用 prefetch 指令处理掉，TLB miss 会真的 stall CPU，影响 ALU 利用率。GEPB内循环一次计算所需数据总大小不能超过 TLB 寻址空间
    - 计算数据要 Packing，防止 TLB miss
    - nc*kc 尽可能大，但不能大过 TLB 寻址空间的一半
    - 按经验是页表大小的一半

#### 其他
BLOCK SIZE的选择是优化最重要的部分，其余的提升性能的操作还有预取，汇编化代码等等，但是具体的提升效果未知，需要进一步的实验。

## [tpoisonooo](https://github.com/tpoisonooo/how-to-optimize-gemm)

这个仓库的特点是，矩阵是按照row major排列的。
- 1. [register关键字](https://github.com/tpoisonooo/how-to-optimize-gemm/blob/16d9914c607c98062b809ac442214fcf0bddc2b5/armv8/MMult_4x4_8.cpp#L48)
    - register关键字是对编译器的一个提示，表明希望编译器将变量存储在处理器的寄存器中，而不是主存中，但是编译器可能会忽略这个提示（如果编译器觉得这样不合适的话）。访问寄存器变量更快。
        - ` register float a_0_0;` 
- 2. [初始化寄存器](https://github.com/tpoisonooo/how-to-optimize-gemm/blob/16d9914c607c98062b809ac442214fcf0bddc2b5/armv8/MMult_4x4_9.cpp#L62)
    - 可以在声明寄存器变量像声明数组一样直接赋值。
        - `float32x4_t c_p0_sum = { 1.5 };`
        - 只是不知道速度和`float32x4_t c_p0_sum = vdupq_n_f32(1.5);`相比起来怎么样。
- 3. dot
    - 对于内积的计算都采取外积叠加的方式进行。
    - 这个方式的一个确定的优点是不需要重复load A或者B，而是streaming A与B的一排（一般是四个元素）。
- 4. 分块
    - 5个前提
        - 1. mc * kc 要小，小到 『 A + B的 nr 列 + C 的 nr 列 』能够一起塞进 cache
        - 2. 如果 1. 被满足，CPU 计算时不再受内存速度的限制，即得到的gflops值就是真实的计算能力
        - 3. A 或 A 的分块只会被加载进 Cache 一次，gemm过程中不会被换入又换出
        - 4. mc 和 kc 要小，小到 『 A + B的 nr 列 + C 的 nr 列 』能够被 TLB 索引，即一定是小于 L2 cache 的。
        - 5. A 或 A 的分块只被加载到 L2 cache 一次
    - 分配内存对齐的数组
    ```cpp
    // malloc alligned array
    float *fastMalloc(int size)
    {  
        void *ptr = 0;  
        int iRet = posix_memalign(&ptr, 64, size * sizeof(float));  
        assert(0 == iRet);  
        return (float*)ptr;
    }
    ```
- 5. 重排
    - 分块一般都伴随着重排。重排后内存连续的好处是显而易见的。
    - 但是重排数组的耗时与重排后的速度提升之间的trade off是怎么样的？以及怎么重排可以让这个耗时被覆盖或者分摊？
        - 对于数据的pack可以通过simd的load和store实现
        - output不需要重排，直接通过指针store到需要位置
    - [`这里给我的启发是数据Pack尽量要在核心计算过程的外部完成`。](https://zhuanlan.zhihu.com/p/326999014#:~:text=%E8%BF%99%E9%87%8C%E7%BB%99%E6%88%91%E7%9A%84%E5%90%AF%E5%8F%91%E6%98%AF%E3%80%8C%E6%95%B0%E6%8D%AEPack%E5%B0%BD%E9%87%8F%E8%A6%81%E5%9C%A8%E6%A0%B8%E5%BF%83%E8%AE%A1%E7%AE%97%E8%BF%87%E7%A8%8B%E7%9A%84%E5%A4%96%E9%83%A8%E5%AE%8C%E6%88%90%E3%80%8D%E3%80%82)
- 6. 预取
    - `__builtin_prefetch(b, 0, 3);`
- 7. load/store C
    - 比较以下两种方式
        - 直接load output，add后store。
        - 和在循环前load output，以达到预取效果，output一直留在寄存器中，计算完之后store。 
        
```c
      // 直接load，然后store
      v24 = vaddq_f32(vld1q_f32(c), v24);
      v25 = vaddq_f32(vld1q_f32(c + ldc), v25);
      v26 = vaddq_f32(vld1q_f32(c + 2 * ldc), v26);
      v27 = vaddq_f32(vld1q_f32(c + 3 * ldc), v27);

      vst1q_f32(c, v24);
      vst1q_f32(c + ldc, v25);
      vst1q_f32(c + 2 * ldc, v26);
      vst1q_f32(c + 3 * ldc, v27);
```
- 8. load A
    - 比较一下两种方式
        - 先vld，再用v_get_lane访问元素
        - 直接通过指针访问元素
